
# 🧬 NanoDockPredictor - Full Version (Google Colab)
### Created by Jaafar (2025)

✅ Smart Docking of Proteins and Ligands  
✅ AutoDock Vina + OpenBabel + Gaussian Input Generator

---

⚠️ Notes:

- This notebook is the **full version** of NanoDockPredictor.
- It runs fully in Google Colab (free).
- No need to install anything on your local machine.

---

Let's start 🚀


In [9]:

# ✅ Install AutoDock Vina and OpenBabel
!apt-get install -y autodock-vina
!apt-get install -y openbabel
!pip install -q requests


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autodock-vina is already the newest version (1.2.3-2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openbabel is already the newest version (3.1.1+dfsg-6ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:

# ✅ Import libraries
import requests
import os
import subprocess
import re
import shutil


In [11]:

# ✅ Create work directory
work_dir = "/content/nanoDocking_run"
os.makedirs(work_dir, exist_ok=True)


In [12]:

# ✅ Download PDB and Ligand functions
def download_pdb(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_path = os.path.join(work_dir, f"{pdb_id}.pdb")
    with open(pdb_path, "w") as f:
        f.write(response.text)
    return pdb_path

def download_ligand(ligand_name):
    cid_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{ligand_name}/cids/TXT"
    cid_response = requests.get(cid_url)
    cid = cid_response.text.strip()
    sdf_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF"
    sdf_response = requests.get(sdf_url)
    sdf_path = os.path.join(work_dir, f"{ligand_name}_{cid}.sdf")
    with open(sdf_path, "w") as f:
        f.write(sdf_response.text)
    pdb_path = sdf_path.replace(".sdf", ".pdb")
    subprocess.run(f"obabel {sdf_path} -O {pdb_path} --addhydrogens", shell=True, check=True)
    return pdb_path


In [13]:

# Example:
pdb_id = "1A3N"  # change as needed
ligand_name = "Letrozole"  # change as needed

receptor_pdb = download_pdb(pdb_id)
ligand_pdb = download_ligand(ligand_name)

print("✅ Files downloaded.")


✅ Files downloaded.


In [14]:

receptor_pdbqt = receptor_pdb.replace(".pdb", ".pdbqt")
ligand_pdbqt = ligand_pdb.replace(".pdb", ".pdbqt")

subprocess.run(f"obabel {receptor_pdb} -O {receptor_pdbqt} --addhydrogens", shell=True, check=True)
subprocess.run(f"obabel {ligand_pdb} -O {ligand_pdbqt} --addhydrogens", shell=True, check=True)

print("✅ Files converted to PDBQT.")


✅ Files converted to PDBQT.


In [22]:
try:
    subprocess.run(vina_command, shell=True, check=True)
    print("✅ Docking completed.")
except subprocess.CalledProcessError as e:
    print(f"❌ Error running vina: {e}")
    try:
        with open("/content/nanoDocking_run/log.txt", "r") as f:
            print(f.read())
    except FileNotFoundError:
        print("⚠️ log.txt not found — vina likely failed before writing log.")
        !ls -lh /content/nanoDocking_run/1A3N.pdbqt
!ls -lh /content/nanoDocking_run/Letrozole_3902.pdbqt
!vina --version
!head -n 20 /content/nanoDocking_run/1A3N.pdbqt
!head -n 20 /content/nanoDocking_run/Letrozole_3902.pdbqt



❌ Error running vina: Command '
vina --receptor /content/nanoDocking_run/1A3N.pdbqt      --ligand /content/nanoDocking_run/Letrozole_3902.pdbqt      --center_x 0 --center_y 0 --center_z 0      --size_x 20 --size_y 20 --size_z 20      --out /content/nanoDocking_run/result.pdbqt      --log /content/nanoDocking_run/log.txt      --exhaustiveness 8
' returned non-zero exit status 1.
⚠️ log.txt not found — vina likely failed before writing log.
-rw-r--r-- 1 root root 50M Jun  8 13:58 /content/nanoDocking_run/1A3N.pdbqt
-rw-r--r-- 1 root root 2.5K Jun  8 13:58 /content/nanoDocking_run/Letrozole_3902.pdbqt
AutoDock Vina v1.2.3
REMARK  Name = /content/nanoDocking_run/1A3N.pdb
REMARK  2024 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: CA_2  and  C_3
REMARK    2  A    between atoms: CA_2  and  CB_5
REMARK    3  A    between atoms: N_8  and  CA_9
REMARK    4  A    between atoms: CA_9  and  C_10
REMARK    5  A    between atoms: CA_9  and  CB_12

In [ ]:

with open(f"{work_dir}/log.txt", "r") as f:
    log_text = f.read()

affinities = re.findall(r"\d+\s+(-\d+\.\d+)", log_text)
print(f"📈 Binding Affinities (kcal/mol): {affinities}")


In [ ]:

ligand_name_clean = ligand_name if ligand_name.strip() != "" else "Unknown Ligand"

gaussian_text = "%mem=200MW\n"
gaussian_text += "%nprocshared=4\n"
gaussian_text += "%chk=temp.chk\n"
gaussian_text += "# opt freq=raman b3lyp/6-311g(d,p) guess=save empiricaldispersion=gd3bj\n"
gaussian_text += "scf=xqc\n\n"
gaussian_text += f"Receptor with Ligand: {ligand_name_clean}\n"

gaussian_path = os.path.join(work_dir, "gaussian_input.txt")
with open(gaussian_path, "w") as f:
    f.write(gaussian_text)

print(f"✅ Gaussian Input generated: {gaussian_path}")
